# Implement ML pipeline Using the AWS Step Functions Data Science SDK

1. [Introduction](#Introduction)
1. [Setup](#setup)
1. [Create Resources](#Create-Resources)
1. [Build a Machine Learning Workflow](#Build-a-Machine-Learning-Workflow)
1. [Run the Workflow](#Run-the-Workflow)
1. [Clean Up](#Clean-Up)

<a id="Introduction"></a>
## 1. Introduction

This notebook describes how to use the AWS Step Functions Data Science SDK to create a machine learning pipeline across data preparation, model training, model evaluation and model register. The defintion of workflow as beflow:

<div align="center"><img width=300 src="images/stepfunctions_graph.png"></div>

<a id="setup"></a>
## 2. Setup

First, we'll need to install and load all the required modules. Then we'll create fine-grained IAM roles for the Lambda, Glue, and Step Functions resources that we will create. The IAM roles grant the services permissions within your AWS environment.

In [1]:
import sys

!{sys.executable} -m pip install --upgrade stepfunctions

### Import the Required Modules

In [2]:
import uuid
import logging
import stepfunctions
import boto3
import sagemaker
from sagemaker import get_execution_role

from sagemaker.amazon.amazon_estimator import image_uris
from sagemaker.inputs import TrainingInput

from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.sklearn.processing import SKLearnProcessor

from sagemaker.s3 import S3Uploader
from stepfunctions import steps
from stepfunctions.steps import TrainingStep, TransformStep
from stepfunctions.inputs import ExecutionInput
from stepfunctions.workflow import Workflow

session = sagemaker.Session()
stepfunctions.set_stream_logger(level=logging.INFO)

region = boto3.Session().region_name
bucket = session.default_bucket()
id = uuid.uuid4().hex

# SageMaker Execution Role
sagemaker_execution_role = (
    sagemaker.get_execution_role()
)

# Create a unique name for the AWS Glue job to be created. If you change the
# default name, you may need to change the Step Functions execution role.
glue_job_prefix = "glue-customer-churn-etl"
glue_job_name = f"{glue_job_prefix}-{id}"

# Create a unique name for the AWS Lambda function to be created. If you change
# the default name, you may need to change the Step Functions execution role.
query_function_prefix = "query-evaluation-result"
query_function_name = f"{query_function_prefix}-{id}"

prefix = 'sagemaker/DEMO-xgboost-customer-churn-connect'

Next, we'll create fine-grained IAM roles for the Lambda, Glue, and Step Functions resources. The IAM roles grant the services permissions within your AWS environment.

### Add permissions to your notebook role in IAM

The IAM role assumed by your notebook requires permission to create and run workflows in AWS Step Functions. If this notebook is running on a SageMaker notebook instance, do the following to provide IAM permissions to the notebook:

1. Open the Amazon [SageMaker console](https://console.aws.amazon.com/sagemaker/). 
2. Select **Notebook instances** and choose the name of your notebook instance.
3. Under **Permissions and encryption** select the role ARN to view the role on the IAM console.
4. Copy and save the IAM role ARN for later use. 
5. Choose **Attach policies** and search for `AWSStepFunctionsFullAccess`.
6. Select the check box next to `AWSStepFunctionsFullAccess` and choose **Attach policy**.

We also need to provide permissions that allow the notebook instance the ability to create an AWS Lambda function and AWS Glue job. We will edit the managed policy attached to our role directly to encorporate these specific permissions:

1. Under **Permisions policies** expand the AmazonSageMaker-ExecutionPolicy-******** policy and choose **Edit policy**.
2. Select **Add additional permissions**. Choose **IAM**  for Service and **PassRole** for Actions.
3. Under Resources, choose **Specific**. Select **Add ARN** and enter `query_training_status-role` for **Role name with path*** and choose **Add**. You will create this role later on in this notebook.
4. Select **Add additional permissions** a second time. Choose **Lambda** for Service, **Write** for Access level, and **All resources** for Resources.
5. Select **Add additional permissions** a final time. Choose **Glue** for Service, **Write** for Access level, and **All resources** for Resources.
6. Choose **Review policy** and then **Save changes**.

### Configure Execution Roles

Create IAM role for StepFunctions.

In [3]:
from setup_iam_roles import create_sfn_role

sfn_role_name = "AmazonSageMaker-StepFunctionsWorkflowExecutionRole"
workflow_execution_role = create_sfn_role(sfn_role_name,
                                         sagemaker_execution_role,
                                         glue_job_prefix)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


Create an IAM role for Glue Job

In [4]:
from setup_iam_roles import create_glue_role

glue_role_name = "AWS-Glue-S3-Bucket-Access"
glue_role_arn = create_glue_role(glue_role_name, bucket)

Create an IAM role for querying evaluation results and model regisry.

In [5]:
from setup_iam_roles import create_lambda_role
import time

query_lambda_role_name = "query-modelregistry-lambda-role"
lambda_role = create_lambda_role(query_lambda_role_name, bucket)

### Prepare the Dataset
This notebook uses the XGBoost algorithm to automate the classification of unhappy customers for telecommunication service providers. The goal is to identify customers who may cancel their service soon so that you can entice them to stay. This is known as customer churn prediction.

The dataset we use is publicly available and was mentioned in the book [Discovering Knowledge in Data](https://www.amazon.com/dp/0470908742/) by Daniel T. Larose. It is attributed by the author to the University of California Irvine Repository of Machine Learning Datasets.

In [6]:
train_prefix = "train"
val_prefix = "validation"
test_prefix = "test"

raw_data = f"s3://{bucket}/{prefix}/input"
processed_data = f"s3://{bucket}/{prefix}/processed"

train_data = f"{processed_data}/{train_prefix}/"
validation_data = f"{processed_data}/{val_prefix}/"
test_data = f"{processed_data}/{test_prefix}/"

Upload data to `S3 Bucket`

In [7]:
!aws s3 cp ../data/churn_processed.csv $raw_data/churn_processed.csv

upload: ../data/churn_processed.csv to s3://sagemaker-us-east-1-822507008821/sagemaker/DEMO-xgboost-customer-churn-connect/input/churn_processed.csv


<a id="Create-Resources"></a>
## 3. Create Resources
In the following steps we'll create the Glue job and Lambda function that are called from the Step Functions workflow.

### Create the AWS Glue Job for Data Preprocessing

In [8]:
glue_script_location = S3Uploader.upload(
    local_path="code/glue_preprocessing.py",
    desired_s3_uri=f"s3://{bucket}/{prefix}/pipeline/glue_preprossing.py",
    sagemaker_session=session,
)

glue_client = boto3.client("glue")

response = glue_client.create_job(
    Name=glue_job_name,
    Description='Prepare data for SageMaker training',
    Role=glue_role_arn,
    ExecutionProperty={
        'MaxConcurrentRuns': 2
    },
    Command={
        'Name': 'glueetl',
        'ScriptLocation': glue_script_location,
    },
    DefaultArguments={
        "--job-bookmark-option": "job-bookmark-enable",
        "--enable-metrics": "",
        "--additional-python-modules": "pyarrow==2,awswrangler==2.9.0,fsspec==0.7.4"
    },
    MaxRetries=0,
    Timeout=60,
    MaxCapacity=10.0,
    GlueVersion='2.0'
)

### Create the AWS Lambda Function for Querying Evalution Result

In [9]:
import zipfile

zip_name = "query_evaluation_result.zip"
lambda_source_code = "./code/query_evaluation_result.py"

zf = zipfile.ZipFile(zip_name, mode="w")
zf.write(lambda_source_code, arcname=lambda_source_code.split("/")[-1])
zf.close()

S3Uploader.upload(
    local_path=zip_name,
    desired_s3_uri=f"s3://{bucket}/{prefix}/lambdaFunctions",
    sagemaker_session=session,
)

's3://sagemaker-us-east-1-822507008821/sagemaker/DEMO-xgboost-customer-churn-connect/lambdaFunctions/query_evaluation_result.zip'

***It takes time to take effect after creating Lambda Role, please run `lambda_client.create_function`again if you come across the following error***
```
InvalidParameterValueException: An error occurred (InvalidParameterValueException) when calling the CreateFunction operation: The role defined for the function cannot be assumed by Lambda.
```

In [10]:
lambda_client = boto3.client("lambda")

response = lambda_client.create_function(
    FunctionName=query_function_name,
    Runtime="python3.7",
    Role=lambda_role,
    Handler="query_evaluation_result.lambda_handler",
    Code={"S3Bucket": bucket, "S3Key": f"{prefix}/lambdaFunctions/{zip_name}"},
    Description="Queries a Processing Job for evalution and return the results.",
    Timeout=15,
    MemorySize=128,
)

### Create the AWS Lambda Function for Model Registry

In [11]:
zip_name = "register_model.zip"
lambda_source_code = "./code/register_model.py"

zf = zipfile.ZipFile(zip_name, mode="w")
zf.write(lambda_source_code, arcname=lambda_source_code.split("/")[-1])
zf.close()

S3Uploader.upload(
    local_path=zip_name,
    desired_s3_uri=f"s3://{bucket}/{prefix}/lambdaFunctions",
    sagemaker_session=session,
)

's3://sagemaker-us-east-1-822507008821/sagemaker/DEMO-xgboost-customer-churn-connect/lambdaFunctions/register_model.zip'

In [12]:
model_register_function_name = "model-register-{}".format(id)
response = lambda_client.create_function(
    FunctionName=model_register_function_name,
    Runtime="python3.7",
    Role=lambda_role,
    Handler="register_model.lambda_handler",
    Code={"S3Bucket": bucket, "S3Key": f"{prefix}/lambdaFunctions/{zip_name}"},
    Description="Register model.",
    Timeout=15,
    MemorySize=128,
)

### Configure the AWS SageMaker Estimator

In [13]:
from sagemaker.workflow.parameters import (
    ParameterFloat,
    ParameterInteger,
    ParameterString,
)

image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.0-1",
    py_version="py3",
)

train_instance_count = 1
train_instance_type = "ml.m5.xlarge"

model_output = f"s3://{bucket}/{prefix}/model"

xgb_train = sagemaker.estimator.Estimator(
    image_uri=image_uri,
    instance_type=train_instance_type,
    instance_count=train_instance_count,
    output_path=model_output,
    base_job_name=f"{prefix}-train",
    sagemaker_session=session,
    role=sagemaker_execution_role,
)

# Set some hyper parameters
# https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html
xgb_train.set_hyperparameters(
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.8,
    silent=0,
    objective="binary:logistic",
    num_round=100,
    eval_metric='auc'
)


<a id="Build-a-Machine-Learning-Workflow"></a>
## 4. Build a Machine Learning Workflow

You can use a state machine workflow to create a model retraining pipeline. The AWS Data Science Workflows SDK provides several AWS SageMaker workflow steps that you can use to construct an ML pipeline. In this tutorial you will create the following steps:

* [**ETLStep**](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/compute.html#stepfunctions.steps.compute.GlueStartJobRunStep) - Starts an AWS Glue job to extract the latest data from our source database and prepare our data.
* [**TrainingStep**](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/sagemaker.html#stepfunctions.steps.sagemaker.TrainingStep) - Creates the training step and passes the defined estimator.
* [**ModelStep**](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/sagemaker.html#stepfunctions.steps.sagemaker.ModelStep) - Creates a model in SageMaker using the artifacts created during the TrainingStep.
* [**LambdaStep**](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/compute.html#stepfunctions.steps.compute.LambdaStep) - Creates the task state step within our workflow that calls a Lambda function.
* [**ChoiceStateStep**](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/states.html#stepfunctions.steps.states.Choice) - Creates the choice state step within our workflow.
* [**EndpointConfigStep**](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/sagemaker.html#stepfunctions.steps.sagemaker.EndpointConfigStep) - Creates the endpoint config step to define the new configuration for our endpoint.
* [**EndpointStep**](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/sagemaker.html#stepfunctions.steps.sagemaker.EndpointStep) - Creates the endpoint step to update our model endpoint.
* [**FailStateStep**](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/states.html#stepfunctions.steps.states.Fail) - Creates fail state step within our workflow.

In [14]:
# SageMaker expects unique names for each job, model and endpoint.
# If these names are not unique the execution will fail.
execution_input = ExecutionInput(
    schema={
        "TrainingJobName": str,
        "GlueJobName": str,
        "QueryLambdaFunctionName": str,
        "RegisterLambdaFunctionName": str,
        "EvaluationStep": str,
        "ModelName": str,
        "EndpointName": str,
        "EndpointConfigName": str
    }
)

### Create an ETL step with AWS Glue
In the following cell, we create a Glue step thats runs an AWS Glue job. The Glue job extracts the latest data from our source database, removes unnecessary columns, splits the data in to training and validation sets, and saves the data to CSV format in S3. Glue is performing this extraction, transformation, and load (ETL) in a serverless fashion, so there are no compute resources to configure and manage. See the [GlueStartJobRunStep](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/compute.html#stepfunctions.steps.compute.GlueStartJobRunStep) Compute step in the AWS Step Functions Data Science SDK documentation.

In [15]:
etl_step = steps.GlueStartJobRunStep(
    "GlueJobPreprocessing",
    parameters={
        "JobName": execution_input["GlueJobName"],
        "Arguments": {
            # Specify any arguments needed based on bucket and keys (e.g. input/output S3 locations)
            '--job-bookmark-option': 'job-bookmark-enable',
            '--additional-python-modules': 'pyarrow==2,awswrangler==2.9.0,fsspec==0.7.4',
            # Custom arguments below
            '--INPUT_DIR': raw_data,
            '--PROCESSED_DIR': processed_data
        },
    },
)

### Create a SageMaker Training Step 

In the following cell, we create the training step and pass the estimator we defined above. See  [TrainingStep](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/sagemaker.html#stepfunctions.steps.sagemaker.TrainingStep) in the AWS Step Functions Data Science SDK documentation to learn more.

In [16]:
train_job_name = "regression-{}".format(id)

training_step = steps.TrainingStep(
    "ModelTraining",
    estimator=xgb_train,
    data={
        "train": TrainingInput(train_data, content_type="text/csv"),
        "validation": TrainingInput(validation_data, content_type="text/csv"),
    },
    job_name=execution_input["TrainingJobName"],
    wait_for_completion=True,
)

### Create a Model Step

In [17]:
model_step = steps.ModelStep(
    "SaveModel",
    model=training_step.get_expected_model(),
    model_name=execution_input["ModelName"],
    result_path="$.ModelStepResults",
)

### Create a ProcessingStep for Evaluation

Upload evaluation code to `S3 Bucket`.

In [18]:
input_evaluation_code = session.upload_data(
    "./code/evaluation.py",
    bucket=bucket,
    key_prefix=f"{prefix}/code",
)

In [19]:
model_data_s3_uri = f"{model_output}/{train_job_name}/output/model.tar.gz"
output_model_evaluation_s3_uri = f"s3://{bucket}/{prefix}/evaluation/"

inputs_evaluation = [
    ProcessingInput(
        source=test_data,
        destination="/opt/ml/processing/test",
        input_name="test-data",
    ),
    ProcessingInput(
        source=model_data_s3_uri,
        destination="/opt/ml/processing/model",
        input_name="model_uri",
    ),
    ProcessingInput(
        source=input_evaluation_code,
        destination="/opt/ml/processing/input/code",
        input_name="code",
    ),
]

outputs_evaluation = [
    ProcessingOutput(
        source="/opt/ml/processing/evaluation",
        destination=output_model_evaluation_s3_uri,
        output_name="evaluation",
    ),
]

In [20]:
model_evaluation_processor = SKLearnProcessor(
    framework_version="0.20.0",
    role=sagemaker_execution_role,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    max_runtime_in_seconds=1200,
)

In [21]:
processing_evaluation_step = steps.ProcessingStep(
    "Evaluation",
    processor=model_evaluation_processor,
    job_name=execution_input["RegisterLambdaFunctionName"],
    inputs=inputs_evaluation,
    outputs=outputs_evaluation,
    container_entrypoint=["python3", "/opt/ml/processing/input/code/evaluation.py"],
)

### Create a Lambda Step For Querying Evaluation Results
In the following cell, we define a lambda step that will invoke the previously created lambda function as part of our Step Function workflow. See [LambdaStep](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/compute.html#stepfunctions.steps.compute.LambdaStep) in the AWS Step Functions Data Science SDK documentation to learn more.

In [22]:
lambda_step = steps.compute.LambdaStep(
    "QueryEvaluationResults",
    parameters={
        "FunctionName": execution_input["QueryLambdaFunctionName"],
        "Payload": {
            "EvaluationResult": output_model_evaluation_s3_uri,
            "S3ModelArtifacts": model_data_s3_uri
        },
    },
)



### Create a Choice State Step 
In the following cell, we create a choice step in order to build a dynamic workflow. This choice step branches based off of the results of our Evaluation step: did the training job fail or should the model be registered into Model Registry? We will add specfic rules to this choice step later on in section 8 of this notebook.

In [23]:
check_accuracy_step = steps.states.Choice("CheckEvaluation")

If the accuracy of model is higher than threshold, you can ether registry the model or deploy model directly. We provide both options in this tutorial, but you may need to choose one of them in your case.

### Create Lambda Step for model registry

In [24]:
model_package_group_name = f"CustomerChurnModelPackage"

model_register_step = steps.compute.LambdaStep(
    "RegisterModel",
    parameters={
        "FunctionName": execution_input["RegisterLambdaFunctionName"],
        "Payload": {
            "S3ModelArtifacts.$": "$.Payload.S3ModelArtifacts",
            "EvaluationResult": output_model_evaluation_s3_uri,
            "ImageUri": image_uri,
            "ModelPackageGroupName": model_package_group_name
        },
    }
)

### Create an Endpoint Configuration Step

In [25]:
endpoint_config_step = steps.EndpointConfigStep(
    "CreateModelEndpointConfig",
    endpoint_config_name=execution_input["EndpointConfigName"],
    model_name=execution_input["ModelName"],
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
)

### Create the Model Endpoint Step

In [26]:
endpoint_step = steps.EndpointStep(
    "CreateModelEndpoint",
    endpoint_name=execution_input["EndpointName"],
    endpoint_config_name=execution_input["EndpointConfigName"],
    update=False,
)

### Create the Fail State Step
In addition, we create a Fail step which proceeds from our choice state if the validation accuracy of our model is lower than the threshold we define. See [FailStateStep](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/states.html#stepfunctions.steps.states.Fail) in the AWS Step Functions Data Science SDK documentation to learn more.

In [27]:
fail_step = steps.states.Fail(
    "Model Accuracy Too Low", comment="Validation accuracy lower than threshold"
)

### Add Rules to Choice State
In the cells below, we add a threshold rule to our choice state. Therefore, if the validation accuracy of our model is below 0.90, we move to the Fail State. If the validation accuracy of our model is above 0.90, we move to the register model step.

In [28]:
threshold_rule = steps.choice_rule.ChoiceRule.NumericGreaterThanEquals(
    variable=lambda_step.output()["Payload"]["trainingMetrics"], value=0.90
)

check_accuracy_step.add_choice(rule=threshold_rule, next_step=model_register_step)
model_register_step.next(endpoint_config_step)
endpoint_config_step.next(endpoint_step)
check_accuracy_step.default_choice(next_step=fail_step)

### Link all the Steps Together
Finally, create your workflow definition by chaining all of the steps together that we've created. See [Chain](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/sagemaker.html#stepfunctions.steps.states.Chain) in the AWS Step Functions Data Science SDK documentation to learn more.

In [29]:
workflow_definition = steps.Chain(
    [etl_step, training_step, model_step, processing_evaluation_step, lambda_step, check_accuracy_step]
)

<a id='Run-the-Workflow'></a>
## 5. Run the Workflow
Create your workflow using the workflow definition above, and render the graph with [render_graph](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Workflow.render_graph):

In [30]:
workflow = Workflow(
    name="MyInferenceRoutine_{}".format(id),
    definition=workflow_definition,
    role=workflow_execution_role,
    execution_input=execution_input,
)

In [31]:
workflow.render_graph()

Create the workflow in AWS Step Functions with [create](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Workflow.create).

Please run this cell again when encounting the following error.
```
ClientError: An error occurred (AccessDeniedException) when calling the CreateStateMachine operation: Neither the global service principal states.amazonaws.com, nor the regional one is authorized to assume the provided role.
```

In [32]:
workflow.create()

[INFO] Workflow created successfully on AWS Step Functions.


'arn:aws:states:us-east-1:822507008821:stateMachine:MyInferenceRoutine_296da389c5134e1ea71c5a963cbea190'

Run the workflow with [execute](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Workflow.execute):

In [33]:
model_name = f"xgb-churn-pred-model-{id}"
endpoint_name = f"xgbChurnPredEndpoint{id}"
endpoint_config_name = f"xgbChurnPredEndpointConf{id}"

execution = workflow.execute(
    inputs={
        "TrainingJobName": train_job_name,  # Each Sagemaker Job requires a unique name,
        "GlueJobName": glue_job_name,
        "QueryLambdaFunctionName": query_function_name,
        "RegisterLambdaFunctionName": model_register_function_name,
        "EvaluationStep": f"processingstep-{id}",
        "ModelName": model_name,
        "EndpointName": endpoint_name,
        "EndpointConfigName": endpoint_config_name
    }
)

[INFO] Workflow execution started successfully on AWS Step Functions.


Render workflow progress with the [render_progress](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Execution.render_progress). This generates a snapshot of the current state of your workflow as it executes. This is a static image therefore you must run the cell again to check progress:

In [34]:
execution.render_progress()

Use [list_executions](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Workflow.list_executions) to list all executions for a specific workflow:

In [35]:
workflow.list_executions(html=True)

Name,Status,Started,End Time
b1219731-3692-4d0d-b393-e37698b6850e,RUNNING,"Jun 19, 2022 03:09:42.578 AM",-


<a id="Clean-Up"></a>
## 6. Clean Up
When you are done, make sure to clean up your AWS account by deleting resources you won't be reusing. Uncomment the code below and run the cell to delete the Glue job, Lambda function, and Step Function.

In [ ]:
from setup_iam_roles import delete_role

workflow.delete()

# Delete sagemaker resources
sagemaker_client = boto3.client('sagemaker')
try:
    sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
except Exception as e:
    print(f'Removing EndpointName {endpoint_name} failed. {e}')

try:
    sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
except Exception as e:
    print(f'Removing EndpointConfigName {endpoint_config_name} failed. {e}')

try:
    sagemaker_client.delete_model(ModelName=model_name)
except Exception as e:
    print(f'Removing ModelName {model_name} failed. {e}')

try:
    sagemaker_client.delete_model_package(ModelPackageName=model_package_group_name)
except Exception as e:
    print(f'Removing ModelPackageName {model_package_group_name} failed. {e}')

# Delete roles
for role_name in [sfn_role_name, glue_role_name, query_lambda_role_name]:
    delete_role(role_name)

# Delete glue job
glue_client.delete_job(JobName=glue_job_name)

# Delete lambda functions
try:
    lambda_client.delete_function(FunctionName=query_function_name)
except Exception as e:
    print(e)

try:
    lambda_client.delete_function(FunctionName=model_register_function_name)
except Exception as e:
    print(e)